# Kubric Utils

In [1]:
import logging

import bpy
import kubric as kb
from kubric.simulator import PyBullet
from kubric.renderer import Blender
import numpy as np

pybullet build time: Mar 26 2022 03:01:35
/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Copyright 2023 The Kubric Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# This method is from the Kubric repo: 
#  > kubricchallenges/movi/movi_c_worker.py
def get_linear_camera_motion_start_end(
    movement_speed: float,
    inner_radius: float = 8.,
    outer_radius: float = 12.,
    z_offset: float = 0.1,
):
  """Sample a linear path which starts and ends within a half-sphere shell."""
  while True:
    camera_start = np.array(kb.sample_point_in_half_sphere_shell(inner_radius,
                                                                 outer_radius,
                                                                 z_offset))
    direction = rng.rand(3) - 0.5
    movement = direction / np.linalg.norm(direction) * movement_speed
    camera_end = camera_start + movement
    if (inner_radius <= np.linalg.norm(camera_end) <= outer_radius and
        camera_end[2] > z_offset):
      return camera_start, camera_end



In [ ]:
# Kubric works with "normalized" image coordinates $u,v \in [0,1]$
# The intrinsic matrix is given by
# ```
#       K =[[f/W   0  -0.5 ]
#           [ 0  -f/H -0.5 ]
#           [ 0    0  -1.0 ]]
# ```
# where W,H are the sensor width and height and f is the focal length. 
# Image coordinates (u,v) and Camera coordinates (x,y,z) are related as follows:
# ```
#       -z  * (u,v,1)^T =  K * (x,y,z)^T
# ```
# ...
# > https://github.com/google-research/kubric/blob/main/kubric/core/cameras.py#L23
def z_to_xyz(z, img_shape, K):
    h, w, _ = z.shape[-3:] 
    uv  = np.stack(np.meshgrid(np.linspace(1,0, w), np.linspace(0,1, h), indexing="ij"), axis=0)
    uv1 = np.concatenate([uv, np.ones((1,w,h))], axis=0)
    z   = data_stack["depth"][t].T
    xyz = np.linalg.inv(K)@(-z*uv1).reshape(3,-1)
    return xyz.reshape(3,w,h).T